# Table of Contents
 <p><div class="lev1"><a href="#Import-Python-modules"><span class="toc-item-num">1 - </span>Import <code>Python</code> modules</a></div><div class="lev1"><a href="#Set-CONSTANTS"><span class="toc-item-num">2 - </span>Set <em>CONSTANTS</em></a></div><div class="lev1"><a href="#Import-TixToGo-data"><span class="toc-item-num">3 - </span>Import TixToGo data</a></div><div class="lev1"><a href="#Create-(Unlevered)-Valuation-Model-Structure"><span class="toc-item-num">4 - </span>Create (Unlevered) Valuation Model Structure</a></div><div class="lev1"><a href="#Valuation-conditional-upon-eventual-success"><span class="toc-item-num">5 - </span>Valuation conditional upon eventual success</a></div><div class="lev1"><a href="#Share-to-give-to-VC-who-targets-50%-success-scenario-IRR-with-5-MM-investment-in-1999"><span class="toc-item-num">6 - </span>Share to give to VC who targets 50% success-scenario IRR with 5 MM investment in 1999</a></div><div class="lev1"><a href="#Capital-Structure-Payoff-Analysis"><span class="toc-item-num">7 - </span>Capital Structure Payoff Analysis</a></div>

# Import `Python` modules

In [1]:
from __future__ import division, print_function
import numpy
import pandas
from pprint import pprint
import xlrd

from CorpFin.Capital import CapitalStructure
from CorpFin.Security import Security
from CorpFin.Valuation import UnlevValModel
from HelpyFuncs.Print import print_live_in_ipython

Using gpu device 0: GeForce GT 750M


In [2]:
pandas.options.display.float_format = '{:,.2f}'.format

In [3]:
print_live_in_ipython()

# Set _CONSTANTS_

In [4]:
# Year 0
YEAR_0 = 1998

# Number of Pro-Forma Years EXCLUDING Year 0
NB_PRO_FORMA_YEARS = 5

# U.S. Corporate Tax rate
US_CORP_TAX_RATE = .4

# CAPM Parameters
RISK_FREE_RATE = .03
PUBIC_MARKET_RISK_PREMIUM = .08
VC_PREMIUM = .08
STARTUP_PERIOD_BETA = 2.
STABILIZED_BETA = 1.

# Long-Term Growth Rate
LONG_TERM_GROWTH_RATE = .04

# Dollar Unit
DOLLAR_UNIT = 1e6   # present financial numbers in $ MMs 

# Import TixToGo data

In [5]:
data_workbook = \
    xlrd.open_workbook(
        filename='data/TixToGo.xlsx')
    
pro_forma_income_statement_worksheet = \
    data_workbook.sheet_by_name('Exhibit 7c')
    
def pro_forma_income_statement_rows(row_numbers):
    if isinstance(row_numbers, (list, tuple)):
        return \
            [pro_forma_income_statement_rows(row_number)
             for row_number in row_numbers]
    else:
        return \
            pro_forma_income_statement_worksheet.row_values(
                rowx=row_numbers,
                start_colx=2,
                end_colx=7)

pro_forma_balance_sheet_worksheet = \
    data_workbook.sheet_by_name('Exhibit 7d')

def pro_forma_balance_sheet_rows(row_numbers):
    if isinstance(row_numbers, (list, tuple)):
        return \
            [pro_forma_balance_sheet_rows(row_number)
             for row_number in row_numbers]
    else:
        return \
            pro_forma_balance_sheet_worksheet.row_values(
                rowx=row_numbers,
                start_colx=1,
                end_colx=6)
    
pro_forma_cash_flow_statement_worksheet = \
    data_workbook.sheet_by_name('Exhibit 7e')

def pro_forma_cash_flow_statement_rows(row_numbers):
    if isinstance(row_numbers, (list, tuple)):
        return \
            [pro_forma_cash_flow_statement_rows(row_number)
             for row_number in row_numbers]
    else:
        return \
            pro_forma_cash_flow_statement_worksheet.row_values(
                rowx=row_numbers,
                start_colx=2,
                end_colx=7)

pro_forma_revenue = \
    map(lambda x: sum(x) / DOLLAR_UNIT,
        zip(*pro_forma_income_statement_rows(range(4, 8))))
    
pro_forma_expenses = \
        map(lambda x: sum(x) / DOLLAR_UNIT,
            zip(*pro_forma_income_statement_rows([10] + range(15, 20, 2))))

pro_forma_net_fixed_assets = \
    map(lambda x: x / DOLLAR_UNIT,
        pro_forma_balance_sheet_rows(12))
    
pro_forma_depreciation = \
    map(lambda x: x / DOLLAR_UNIT,
        pro_forma_cash_flow_statement_rows(7))
    
pro_forma_net_working_capital_balance = \
    map(lambda x: (sum(x[:2]) - sum(x[2:])) / DOLLAR_UNIT,
        zip(*pro_forma_balance_sheet_rows(range(6, 8) + range(17, 23))))

# Create (Unlevered) Valuation Model Structure

In [6]:
model = \
    UnlevValModel(
        venture_name='TixToGo',
        year_0=YEAR_0,
        nb_pro_forma_years_excl_0=NB_PRO_FORMA_YEARS,
        val_all_years=True)

Compiling:
    PublicMarketPremium... done after 0:00:00 (0:00:00 so far)
    Revenue... done after 0:00:00 (0:00:00 so far)
    RevenueChange... done after 0:00:00 (0:00:00 so far)
    RevenueGrowth... done after 0:00:00 (0:00:01 so far)
    OpEx... done after 0:00:00 (0:00:01 so far)
    OpEx_over_Revenue... done after 0:00:00 (0:00:01 so far)
    OpExGrowth... done after 0:00:00 (0:00:01 so far)
    EBIT... done after 0:00:00 (0:00:02 so far)
    EBITMargin... done after 0:00:00 (0:00:02 so far)
    EBITGrowth... done after 0:00:00 (0:00:03 so far)
    TaxLoss... done after 0:00:00 (0:00:04 so far)
    TaxableEBIT... done after 0:00:00 (0:00:04 so far)
    EBIAT... done after 0:00:00 (0:00:05 so far)
    FA... done after 0:00:00 (0:00:05 so far)
    FA_over_Revenue... done after 0:00:00 (0:00:05 so far)
    FAGrowth... done after 0:00:00 (0:00:06 so far)
    Depreciation... done after 0:00:00 (0:00:07 so far)
    Depreciation_over_prevFA... done after 0:00:00 (0:00:08 so far)
    Ca

# Valuation conditional upon eventual success

In [7]:
common_inputs = \
    dict(
        Revenue=[0.] + pro_forma_revenue,
        OpEx=[0.] + pro_forma_expenses,
        CorpTaxRate=US_CORP_TAX_RATE,
        FA=[0.] + pro_forma_net_fixed_assets,
        Depreciation=[0.] + pro_forma_depreciation,
        NWC=[0.] + pro_forma_net_working_capital_balance,
        RiskFreeRate=RISK_FREE_RATE,
        PublicMarketPremium=PUBIC_MARKET_RISK_PREMIUM,
        StabilizedBeta=STABILIZED_BETA,
        LongTermGrowthRate=LONG_TERM_GROWTH_RATE,
        ProFormaPeriodBeta=STARTUP_PERIOD_BETA)

non_vc_outputs = \
    ['RiskFreeRate', 'PublicMarketPremium',
     ' ',
     'EBIT & EBIAT:',
     'Revenue', 'RevenueGrowth',
     '  ',
     'OpEx',
     '   ',
     'EBIT', 'EBITMargin',
     '    ',
     'TaxLoss', 'TaxableEBIT', 'CorpTaxRate', 'EBIAT',
     '     ',
     'NET FIXED ASSETS, CAPEX & DEPRECIATION:',
     'FA', 'CapEx', 'Depreciation', 'Depreciation_over_prevFA',
     '      ',
     'NET WORKING CAPITAL:',
     'NWC', 'NWC_over_Revenue', 'NWCChange',
     '       ',
     'FREE CASH FLOWS & TERMINAL VALUE:',
     'FCF',
     'StabilizedBeta', 'StabilizedDiscountRate', 'LongTermGrowthRate',
     'TV', 'TV_RevenueMultiple', 'TV_EBITMultiple',
     '        ',
     'Non-VC VALUATION Projection:',
     'ProFormaPeriodBeta', 'ProFormaPeriodDiscountRate', 'Unlev_Val']
    
non_vc_val_results = \
    model(
        outputs=non_vc_outputs,
        **common_inputs)
    
print()

vc_val_results = \
    model(
        outputs=['ProFormaPeriodDiscountRate', 'Unlev_Val'],
        InvestmentManagerFeePremium=VC_PREMIUM,
        **common_inputs)
    
val_results_df = non_vc_val_results['data_frame'].copy()
val_results_df['         '] = ''
val_results_df['VC VALUATION Projection:'] = ''
val_results_df['VC_ProFormaPeriodDiscountRate'] = \
    [vc_val_results['ProFormaPeriodDiscountRate']] + 5 * ['']
val_results_df['VC_Unlev_Val'] = vc_val_results['Unlev_Val']

print('\nVALUATION ($ MM) as at Early 1999, Conditional Upon Eventual Success')
val_results_df.T

Calculating:
    PublicMarketPremium
    Revenue
    RevenueGrowth
    OpEx
    EBIT
    EBITMargin
    TaxLoss
    TaxableEBIT
    EBIAT
    FA
    CapEx
    Depreciation
    Depreciation_over_prevFA
    NWC
    NWC_over_Revenue
    NWCChange
    FCF
    StabilizedDiscountRate
    TV
    TV_RevenueMultiple
    TV_EBITMultiple
    ProFormaPeriodDiscountRate
    Unlev_Val
done!

Calculating:
    ProFormaPeriodDiscountRate
    Unlev_Val
done!

VALUATION ($ MM) as at Early 1999, Conditional Upon Eventual Success


,Year 0,1999,2000,2001,2002,2003
RiskFreeRate,0.03,,,,,
PublicMarketPremium,0.08,,,,,
,,,,,,
EBIT & EBIAT:,,,,,,
Revenue,0.00,0.03,21.98,57.32,121.45,187.38
RevenueGrowth,NaN,0.00,743.29,1.61,1.12,0.54
,,,,,,
OpEx,0.00,0.98,17.75,43.40,81.00,119.64
,,,,,,
EBIT,0.00,-0.95,4.23,13.92,40.45,67.75


# Share to give to VC who targets 50% success-scenario IRR with 5 MM investment in 1999

In [8]:
vc_investment = 5.

terminal_value_2003 = non_vc_val_results['TV']

In [9]:
vc_share_early_1999 = .057

holding_period = 5

vc_irr = numpy.irr(
    [-vc_investment] +
    (holding_period - 1) * [0.] +
    [vc_share_early_1999 * terminal_value_2003])

print("VC's Investment in Early 1999               =   $%.0f MM\n"
      % vc_investment)
print("VC's Share in TixToGo                       =   %s\n"
      % '{:.1%}'.format(vc_share_early_1999))
print("VC's Holding Period                         =   %d Years\n"
      % holding_period)
print("VC's Share in $%.2f MM Exit Value in 2003 =   $%.2f MM\n"
      % (terminal_value_2003, vc_share_early_1999 * terminal_value_2003))
print("VC's IRR                                    =   %s"
      % '{:.1%}'.format(vc_irr))

VC's Investment in Early 1999               =   $5 MM

VC's Share in TixToGo                       =   5.7%

VC's Holding Period                         =   5 Years

VC's Share in $672.44 MM Exit Value in 2003 =   $38.33 MM

VC's IRR                                    =   50.3%


In [10]:
vc_share_late_1999 = .038

holding_period = 4

vc_irr = numpy.irr(
    [-vc_investment] +
    (holding_period - 1) * [0.] +
    [vc_share_late_1999 * terminal_value_2003])

print("VC's Investment in Late 1999                =   $%.0f MM\n"
      % vc_investment)
print("VC's Share in TixToGo                       =   %s\n"
      % '{:.1%}'.format(vc_share_late_1999))
print("VC's Holding Period                         =   %d Years\n"
      % holding_period)
print("VC's Share in $%.2f MM Exit Value in 2003 =   $%.2f MM\n"
      % (terminal_value_2003, vc_share_late_1999 * terminal_value_2003))
print("VC's IRR                                    =   %s"
      % '{:.1%}'.format(vc_irr))

VC's Investment in Late 1999                =   $5 MM

VC's Share in TixToGo                       =   3.8%

VC's Holding Period                         =   4 Years

VC's Share in $672.44 MM Exit Value in 2003 =   $25.55 MM

VC's IRR                                    =   50.4%


# Capital Structure Payoff Analysis

In [11]:
capital_structure = \
    CapitalStructure(
        Security(
            label='Common'),
        [[Security(
            label='Series B Conv Pref',
            claim_val=5000.),
         1.]],
        Security(
            label='Series C Partg Pref: 1st Dip',
            claim_val=15e6 / 2160))

print('Security Classes:')
capital_structure.show()

Security Classes:


,Liquidation Order (LIFO),Outstanding,Conversion Ratio
Common,0,0.00,nan
Series B Conv Pref,1,0.00,1.00
Series C Partg Pref: 1st Dip,2,0.00,nan


The Founders hold 2,000 Common Shares:

In [12]:
capital_structure.issue(
    owner='Founders',
    securities=[(2000, 'Common')])

capital_structure.show(ownerships=True)

,Owner,Security,Quantity
0,Founders,Common,"2,000.00"


The Series B Holders hold 1,600 Convertible Preferred Shares:

In [13]:
capital_structure.issue(
    owner='Series B Holders',
    securities=[(1600, 'Series B Conv Pref')])

capital_structure.show(ownerships=True)

,Owner,Security,Quantity
0,Founders,Common,"2,000.00"
1,Series B Holders,Series B Conv Pref,"1,600.00"


The Series C Holders hold 2,160 Participating Preferred Shares, which are equivalent to 2,160 "first-dip" debt notes plus 2,160 Common Shares: 

In [14]:
capital_structure.issue(
    owner='Series C Holders',
    securities=[
        (2160, 'Common'),
        (2160, 'Series C Partg Pref: 1st Dip')])

capital_structure.show(ownerships=True)

,Owner,Security,Quantity
0,Founders,Common,"2,000.00"
1,Series C Holders,Series C Partg Pref: 1st Dip,"2,160.00"
2,Series C Holders,Common,"2,160.00"
3,Series B Holders,Series B Conv Pref,"1,600.00"


Let's now examine the payoffs to the capital holders over a range of Enterprise Valuations:

In [15]:
def show_payoffs(enterprise_val):

    print('ENTERPRISE VAL =  $%s MM' % '{:,.2f}'.format(enterprise_val / 1e6))
    pprint(
        pandas.Series(
            data=capital_structure.val(
                enterprise_val=enterprise_val,
                pareto_equil_conversions=True)['ownership_vals']) / 1e6)

    print('\nDETAILS:')
    details_df = \
        capital_structure(
            enterprise_val=enterprise_val,
            pareto_equil_conversions=True,
            ownerships=True)
    details_df.Val /= 1e6
    pprint(details_df)

In [16]:
show_payoffs(30e6)

ENTERPRISE VAL =  $30.00 MM
Founders            3.37
Series B Holders    8.00
Series C Holders   18.63
dtype: float64

DETAILS:
                  Owner                      Security   Val  Share
0              Founders                        Common  3.37   0.48
1      Series C Holders  Series C Partg Pref: 1st Dip 15.00    nan
2      Series C Holders                        Common  3.63   0.52
3      Series B Holders            Series B Conv Pref  8.00    nan
TOTAL                                                 30.00   1.00


In [17]:
show_payoffs(40e6)

ENTERPRISE VAL =  $40.00 MM
Founders            8.17
Series B Holders    8.00
Series C Holders   23.83
dtype: float64

DETAILS:
                  Owner                      Security   Val  Share
0              Founders                        Common  8.17   0.48
1      Series C Holders  Series C Partg Pref: 1st Dip 15.00    nan
2      Series C Holders                        Common  8.83   0.52
3      Series B Holders            Series B Conv Pref  8.00    nan
TOTAL                                                 40.00   1.00


In [18]:
show_payoffs(50e6)

ENTERPRISE VAL =  $50.00 MM
Founders           12.15
Series B Holders    9.72
Series C Holders   28.12
dtype: float64

DETAILS:
                  Owner                      Security   Val  Share
0              Founders                        Common 12.15   0.35
1      Series C Holders  Series C Partg Pref: 1st Dip 15.00    nan
2      Series C Holders                        Common 13.12   0.38
3      Series B Holders                        Common  9.72   0.28
TOTAL                                                 50.00   1.00


In [19]:
show_payoffs(60e6)

ENTERPRISE VAL =  $60.00 MM
Founders           15.62
Series B Holders   12.50
Series C Holders   31.87
dtype: float64

DETAILS:
                  Owner                      Security   Val  Share
0              Founders                        Common 15.62   0.35
1      Series C Holders  Series C Partg Pref: 1st Dip 15.00    nan
2      Series C Holders                        Common 16.88   0.38
3      Series B Holders                        Common 12.50   0.28
TOTAL                                                 60.00   1.00
